In [1]:
from IPython.core.display import HTML, display
HTML("<style>.container { width:100% !important; }</style>")
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [2]:
import operator
#import pandas as pd
import numpy as np
from Data.sentiment_dict import positive, negative
#from sklearn.feature_extraction import DictVectorizer
from nltk.corpus import stopwords
#pd.options.display.max_colwidth=250

In [30]:
# separate tweet into parts with emojis of interest and the rest
def contain_emoji(tweet, emojis):
    tweet_emoji = set(tweet).intersection(emojis)
    if len(tweet_emoji) == 0:
        return False
    return True

def filter_emoji(tweet, emojis):
    # Return the text without emojis.
    text = ""
    for i in tweet:
        if i not in emojis:
            text += i
    return text

def filter_stop(word_list, stop_words):
    # Return the words that are not contained in 'stop_words'.
    return [word for word in word_list if word.lower() not in stop_words]

def convert_to_ngrams(text, n):    # num_ngrams = number of ngrams given user-defined n  e.g. n=1 unigrams; n=2 bigrams
    # Convert text to a list of words.
    word_list = text.split()
    num_ngrams=len(word_list) - n + 1
    ngram_list = []
    # i = location of ngram's starting word in word list / tweet
    for i in range (0, num_ngrams):
        # Construct the ngram by chaining the words together.
        ngram = word_list[i]
        # j is the location of word in the ngram.
        for j in range (1, n):
            ngram += " " + word_list[i + j]
        ngram_list.append(ngram)
    return ngram_list

def update_ngram_count(text, stop_words, n, ngram_count):
    # Update 'ngram_count' dictionary with the frequency ngrams that present in 'text'. 'n' is the
    # degree of ngrams. If 'n' is 1, then we will filter the unigram using 'stop_words'.
    ngram_list = convert_to_ngrams(text, n)
    # Only filter stop words for unigrams i.e. ngram_n=1
    if n == 1:
        ngram_list = filter_stop(ngram_list, stop_words)
    for ngram in set(ngram_list):
        if ngram not in ngram_count:
            ngram_count[ngram] = 1
        else: 
            ngram_count[ngram] += 1
                    
    

In [31]:
# specify the location of the tweets
negtweet_file ='./Data/negtweets.txt'
postweet_file ='./Data/postweets.txt'

In [39]:
# The words in the tweets that contain positive emoji and their count. We only count the word once even if the word appears
# multiple times in a tweet.
positive_ngram_count = {}
# The words in the tweets that contain positive emoji and their count. We only count the word once even if the word appears
# multiple times in a tweet.
negative_ngram_count = {}

all_emojis = positive + negative
#stop_words=stopwords.words("english")
stop_words = set(stopwords.words("english"))
added_stop_words = ["i'm", "it's"]
stop_words.update(added_stop_words)

ii = 0
ngram_n = 4
# load all the tweets from negative file
with open(postweet_file,'r') as f:
    for line in f:
        # For testing purpose, only process a limited number of tweets in the file.
        #if ii >= 200:
         # break
        #ii = ii +1
        
        tweet = line.decode('utf-8')
        text_without_emoji = filter_emoji(tweet, all_emojis)
        # Process the tweet that contains positive emoji.
        if contain_emoji(tweet, positive):
            update_ngram_count(text_without_emoji, stop_words, ngram_n, 
                               positive_ngram_count)
                    
        # Process the tweet that contains negative emoji.            
        if contain_emoji(tweet, negative):
            update_ngram_count(text_without_emoji, stop_words, ngram_n, 
                               negative_ngram_count)

# load all the tweets from negative file
ii = 0
with open(negtweet_file,'r') as f:
    for line in f:
        # For testing purpose, only process a limited number of tweets in the file.
        # if ii >= 200:
        #    break
        #ii = ii +1
        
        tweet = line.decode('utf-8')
        text_without_emoji = filter_emoji(tweet, all_emojis)
        
         # Process the tweet that contains positive emoji.
        if contain_emoji(tweet, positive):
            update_ngram_count(text_without_emoji, stop_words, ngram_n, 
                               positive_ngram_count)
                    
        # Process the tweet that contains negative emoji.            
        if contain_emoji(tweet, negative):
            update_ngram_count(text_without_emoji, stop_words, ngram_n, 
                               negative_ngram_count)
    
# Sort the positive and negative words based on their counts.
# sorted in descending order using "reverse = true"
sorted_positive_ngrams = sorted(positive_ngram_count.items(), key=operator.itemgetter(1), reverse=True)
#print "\nSorted Positive Word Count"
#print sorted_positive_words
print ("Top 50 Positive Ngrams, where n=", ngram_n)
ii = 0
for ngram in sorted_positive_ngrams: 
    if ii >= 50:
        break
    ii +=1
    print ngram

sorted_negative_ngrams = sorted(negative_ngram_count.items(), key=operator.itemgetter(1), reverse=True)
print ("\nTop 50 Negative Ngrams, where n=%d" % ngram_n)
ii = 0
for ngram in sorted_negative_ngrams: 
    if ii >= 50:
        break
    ii +=1
    print ngram

#normalize by odds ratio
#use mike's new data set

('Top 50 Positive Ngrams, where n=', 4)
(u'I love you so', 70)
(u'love you so much', 52)
(u"watch 's new video", 29)
(u"Go watch 's new", 23)
(u"can't wait to see", 18)
(u'I love you too', 17)
(u"go watch 's new", 17)
(u"I can't wait to", 14)
(u'IT A LIKE &amp;', 14)
(u'i love you so', 13)
(u"WATCH 's NEW VIDEO", 12)
(u"You Don't Have To", 11)
(u"Don't Have To Go", 11)
(u'Can you follow me', 11)
(u'GIVE IT A LIKE', 11)
(u'love you so so', 10)
(u"I'm so happy for", 10)
(u'it that night\U0001f525\U0001f525 Everybody', 10)
(u'night\U0001f525\U0001f525 Everybody $1 til', 10)
(u'SPIKED it that night\U0001f525\U0001f525', 10)
(u'video \u25b6 [ ]', 10)
(u'will find out who', 10)
(u'who SPIKED it that', 10)
(u'#WhoSpikedThePunch is Back Feb.', 10)
(u'Everybody $1 til 11:30', 10)
(u'that night\U0001f525\U0001f525 Everybody $1', 10)
(u'find out who SPIKED', 10)
(u'Have To Go by', 10)
(u'I LOVE YOU SO', 10)
(u'out who SPIKED it', 10)
(u"Y'all will find out", 10)
(u"I'm so proud of", 9)
(u"'s New 